In [1]:
%%javascript
$('#appmode-leave').hide()
// Hides the edit app button.
$('#appmode-busy').hide()
// Hides the kernel busy indicator.
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false
    // disable scrolling
}

<IPython.core.display.Javascript object>

In [2]:
# Google Forms / Sheets Code
import urllib.request
from bs4 import BeautifulSoup
import requests
import warnings

def get_questions(in_url):
    res = urllib.request.urlopen(in_url)
    soup = BeautifulSoup(res.read(), 'html.parser')

    def get_names(f): return [v for k, v in f.attrs.items() if 'label' in k]

    def get_name(f): return get_names(f)[0] if len(
        get_names(f)) > 0 else 'unknown'
    all_questions = soup.form.findChildren(
        attrs={'name': lambda x: x and x.startswith('entry.')})
    return {get_name(q): q['name'] for q in all_questions}


def submit_response(form_url, cur_questions, verbose=False, **answers):
    submit_url = form_url.replace('/viewform', '/formResponse')
    form_data = {'draftResponse': [],
                 'pageHistory': 0}
    for v in cur_questions.values():
        form_data[v] = ''
    for k, v in answers.items():
        if k in cur_questions:
            form_data[cur_questions[k]] = v
        else:
            warnings.warn('Unknown Question: {}'.format(k), RuntimeWarning)
    if verbose:
        print(form_data)
    user_agent = {'Referer': form_url,
                  'User-Agent': "Mozilla/5.0 (X11; Linux i686) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1500.52 Safari/537.36"}
    return requests.post(submit_url, data=form_data, headers=user_agent)

In [3]:
import os
import ipywidgets as ipw
from glob import glob
import json
import pandas as pd
import numpy as np
from PIL import Image
from itertools import cycle
from io import BytesIO
from time import time
from collections import namedtuple
from IPython.display import display
USERNAME = os.environ.get('APPMODE_USER', 'Not logged in')
HOSTNAME = os.environ.get('HOSTNAME', 'anon')
BINARY_TASK = True
UNKNOWN_OPTION = None

/srv/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/srv/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
with open('task.json', 'r') as f:
    annotation_task = json.load(f)
    annot_df = pd.DataFrame(annotation_task['dataset']['dataframe'])
print('Loaded annotation task', annot_df.shape[0], 'images')

Loaded annotation task 100 images


In [5]:
image_path_dict = {c_path: os.path.join(annotation_task['dataset']['base_image_directory'],
                                        c_path)
                   for c_path in annot_df[annotation_task['dataset']['image_path']]}
category_ids = sorted(
    annot_df[annotation_task['dataset']['output_labels']].unique().tolist())
if (UNKNOWN_OPTION is not None) and not BINARY_TASK:
    category_ids += [UNKNOWN_OPTION]

In [6]:
MultipleChoiceAnswer = namedtuple(
    'MultipleChoiceAnswer', ['answer', 'question'])
TaskResult = namedtuple(
    'TaskResult', ['annotation_mode', 'task', 'item_id', 'label', 'viewing_info'])


class SimpleImageViewer:
    def __init__(self):
        self.cur_image_view = ipw.Image(layout=ipw.Layout(width="512px"),
                                        disabled=True)
        self.loaded_time = None

    def load_image_path(self, path):
        c_img = Image.open(path)
        bio_obj = BytesIO()
        c_img.save(bio_obj, format='png')
        bio_obj.seek(0)
        self.cur_image_view.value = bio_obj.read()
        self.loaded_time = time()

    def get_viewing_info(self):
        out_info = {}
        if self.loaded_time is not None:
            out_info['viewing_time'] = time()-self.loaded_time
        return json.dumps(out_info)

    def get_widget(self):
        return self.cur_image_view


class MultipleChoiceQuestion:
    def __init__(self, question, labels, question_prefix='', width="150px", buttons_per_row=3):
        self.question_box = ipw.HTML(value='')
        self.labels = labels
        self.question_prefix = question_prefix
        self.set_question(question)
        self.width = width
        self.buttons_per_row = buttons_per_row
        self._make_buttons(labels)
        self.submit_func = None

    def on_submit(self, submit_func):
        self.submit_func = submit_func

    def set_question(self, question):
        self.question = question
        self.question_box.value = '<h2>{} {}?</h2>'.format(
            self.question_prefix, self.question)

    def mk_btn(self, description):
        btn = ipw.Button(description=description,
                         layout=ipw.Layout(width=self.width))
        if description == 'Yes':
            btn.style.button_color = 'lightgreen'
        elif description == 'No':
            btn.style.button_color = 'pink'

        def on_click(btn):
            if btn is not None and self.submit_func is not None:
                self.submit_func(MultipleChoiceAnswer(
                    answer=btn.description, question=self.question))
        btn.on_click(on_click)
        return btn

    def get_widget(self):
        return ipw.VBox([self.question_box]+self.button_rows)

    def _make_buttons(self, button_ids):
        self.button_rows = []
        c_row = []
        for i, but_name in enumerate(button_ids, 1):
            c_row += [self.mk_btn(but_name)]
            if (i % self.buttons_per_row) == 0:
                self.button_rows += [ipw.HBox(c_row)]
                c_row = []
        self.button_rows += [ipw.HBox(c_row)]


class ClassificationMixin:
    def __init__(self, labels, images, seed=None):
        self.labels = labels
        self.images = images
        self.image_keys = sorted(list(images.keys()))
        self.submit_event = None
        self.task_widget = SimpleImageViewer()
        self.answer_widget.on_submit(lambda x: self._local_submit(x))
        self.current_image_id = None
        self.set_seed(seed)
        self._local_submit(MultipleChoiceAnswer(question='', answer=''))

    def get_viewing_info(self):
        return self.task_widget.get_viewing_info()

    def get_widget(self):
        return ipw.HBox([self.task_widget.get_widget(), self.answer_widget.get_widget()])

    def set_seed(self, seed):
        if seed is not None:
            np.random.seed(seed)

    def _ipython_display_(self):
        display(self.get_widget())

    def on_submit(self, on_submit):
        self.submit_event = on_submit

    def _submit(self, mc_answer):
        raise NotImplementedError(
            "Subclass needs to have its own implementation of _submit")

    def _update_image(self, image_key):
        # update image
        self.task_widget.load_image_path(self.images[image_key])
        self.current_image_id = image_key

    def _local_submit(self, mc_answer):
        generic_info = dict(item_id=self.current_image_id,
                            viewing_info=self.get_viewing_info())
        c_task_info = self._submit(mc_answer)
        # submit results to backend
        c_task = TaskResult(**generic_info,
                            **c_task_info)

        if self.submit_event is not None:
            self.submit_event(c_task)


class MultiClassTask(ClassificationMixin):
    def __init__(self, labels, images, seed=None):
        self.answer_widget = MultipleChoiceQuestion(
            'Select the most appropriate label for the given image', labels)
        super().__init__(labels, images, seed)

    def _submit(self, mc_answer):
        c_task_dict = dict(annotation_mode='MultiClass',
                           task=','.join(self.labels),
                           label=mc_answer.answer
                           )

        # get next question
        image_key = np.random.choice(self.image_keys)
        self._update_image(image_key)
        return c_task_dict


class BinaryClassTask(ClassificationMixin):
    def __init__(self, labels, images, unknown_option, seed=None):
        answer_choices = ['Yes', 'No']
        if unknown_option is not None:
            q_options.append(unknown_option)
        self.answer_widget = MultipleChoiceQuestion('',
                                                    answer_choices,
                                                    question_prefix='Does this patient have',
                                                    buttons_per_row=1)
        super().__init__(labels, images, seed)

    def _submit(self, mc_answer):
        c_task_dict = dict(annotation_mode='BinaryClass',
                           task=mc_answer.question,
                           label=mc_answer.answer)

        # get next question
        image_key = np.random.choice(self.image_keys)
        question = np.random.choice(self.labels)
        self.answer_widget.set_question(question)

        # update image
        self._update_image(image_key)
        return c_task_dict

In [7]:
FORM_URL = annotation_task['google_forms']['form_url']
anno_questions = get_questions(FORM_URL)


def submit_label(task_result):
    """
    the connection to the backend
    """
    submit_response(FORM_URL, anno_questions,
                    annotator=USERNAME,
                    session=HOSTNAME,
                    time=time(),
                    item_id=task_result.item_id,
                    label=task_result.label,
                    task=task_result.task,
                    annotation_mode=task_result.annotation_mode,
                    viewing_info=task_result.viewing_info
                    )

In [8]:
title_box = ipw.HTML(value=f'<h1> Welcome {USERNAME}</h1>')
if BINARY_TASK:
    task_widget = BinaryClassTask(category_ids, image_path_dict, unknown_option=UNKNOWN_OPTION)
else:
    task_widget = MultiClassTask(category_ids, image_path_dict)
task_widget.on_submit(submit_label)
ipw.VBox([title_box, task_widget.get_widget()])